In [ ]:
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import pdb
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount('/content/drive')
import os
from os import listdir
import csv
from PIL import Image
import tensorflow as tf

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Face cropping method from https://datagen.tech/guides/face-recognition/face-detection-with-opencv-2-quick-tutorials/#:~:text=DNN%20Face%20Detector%20is%20a,scales%2C%20poses%2C%20and%20occlusions.
# Iterating through images in a folder from https://www.geeksforgeeks.org/how-to-iterate-through-images-in-a-folder-python/
# Training/Testing Model from https://www.atmosera.com/blog/facial-recognition-with-cnns/#:~:text=One%20of%20the%20primary%20benchmarks,99%25%20accuracy%20on%20the%20dataset
# This code was run on personal machine and the cropped images were copied into subfolders on google drive to avoid issues with folder size and google colab


# Load the DNN Face Detector model
from cv2 import dnn
prototxt = '/content/drive/MyDrive/ECE50024Challenge/deploy.prototxt.txt'
caffemodel='/content/drive/MyDrive/ECE50024Challenge/res10_300x300_ssd_iter_140000.caffemodel'
face_detector =  cv2.dnn.readNetFromCaffe( prototxt, caffemodel)
#face_detector = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")

# get the path/directory
folder_dir = "/content/drive/MyDrive/ECE50024Challenge/train_small/train_small/"
for images in os.listdir(folder_dir):
    # check if the image ends with jpg
    if (images.endswith(".jpg")):
        #Make images readable
        img=Image.open('/content/drive/MyDrive/ECE50024Challenge/train_small/train_small/'+images)
        imgCheck = cv2.imread('/content/drive/MyDrive/ECE50024Challenge/train_small/train_small/'+images)
        if img.mode != 'RGB' or imgCheck is None:
            img = img.convert(mode="RGB", colors=24)
            img.save('/content/drive/MyDrive/ECE50024Challenge/train_small_readable/'+images)
            img = cv2.imread('/content/drive/MyDrive/ECE50024Challenge/train_small_readable/'+images)
        else:
            img = cv2.imread('/content/drive/MyDrive/ECE50024Challenge/train_small/train_small/'+images)

        # Get the height and width of the input image
        (h, w) = img.shape[:2]

        # Preprocess the image by resizing it and converting it to a blob
        blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

        # Feed the blob as input to the DNN Face Detector model
        face_detector.setInput(blob)
        detections = face_detector.forward()

        # Loop over the detections and draw a rectangle around each face
        confidence = detections[0, 0, 0, 2]

        # Filter out weak detections
        if confidence > 0.25:
            # Get the bounding box for the face
            box = detections[0, 0, 0, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            if startX > np.shape(img)[1] or startX < 0:
                startX = 0
            if startY > np.shape(img)[0] or startY < 0:
                startY = 0
            if endX > np.shape(img)[1] or endX < 0:
                endX = np.shape(img)[1]
            if endY > np.shape(img)[0] or endY < 0:
                endY = np.shape(img)[0]

            faces = img[startY:endY, startX:endX]

            #write cropped faces to separate folder
            faces = cv2.resize(faces, (300, 300)) #resize the images to the same size
            path = '/content/drive/MyDrive/ECE50024Challenge/train_faces'
            cv2.imwrite(os.path.join(path , images), faces)
            cv2.waitKey(0)
        else:
            startX = 0
            startY = 0
            endX = np.shape(img)[1]
            endY = np.shape(img)[0]
            faces = img[startY:endY, startX:endX]

            #write cropped faces to separate folder
            faces = cv2.resize(faces, (300, 300)) #resize the images to the same size
            path = '/content/drive/MyDrive/ECE50024Challenge/train_faces'
            cv2.imwrite(os.path.join(path , images), faces)
            cv2.waitKey(0)

4224.jpg
3749.jpg
6619.jpg
4032.jpg
596.jpg
6220.jpg
554.jpg
2456.jpg
144.jpg
385.jpg
178.jpg
2871.jpg
393.jpg
3164.jpg
1217.jpg
3950.jpg
1828.jpg
6435.jpg


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


1207.jpg
1617.jpg
1830.jpg
3570.jpg
6150.jpg
6636.jpg
5895.jpg
746.jpg
3868.jpg
6535.jpg
5954.jpg
2949.jpg
3128.jpg
3869.jpg
3110.jpg
3306.jpg
1934.jpg
4497.jpg
709.jpg
6135.jpg
2435.jpg
6269.jpg
508.jpg
11.jpg
1645.jpg
3091.jpg
4113.jpg
1254.jpg
6275.jpg
4884.jpg
2173.jpg
2368.jpg
1104.jpg
2425.jpg
296.jpg
2790.jpg
6270.jpg
3081.jpg
2837.jpg
857.jpg
5380.jpg
4500.jpg
6129.jpg
107.jpg
3737.jpg
3093.jpg
847.jpg
106.jpg
6823.jpg
400.jpg
2099.jpg
3423.jpg
2339.jpg
832.jpg
3780.jpg
4945.jpg
5044.jpg
5939.jpg
3187.jpg
4993.jpg
1151.jpg
4798.jpg
6944.jpg
4002.jpg
2881.jpg
5491.jpg
2704.jpg
5889.jpg
2884.jpg
4007.jpg
5682.jpg
6638.jpg


In [ ]:
#Model
input_shape = (64, 64, 3)
num_classes = 100
model = keras.Sequential([
  layers.Rescaling(1./255, input_shape=input_shape),

  layers.Conv2D(32, (3, 3), activation='relu'),
  layers.MaxPooling2D(2, 2),
  layers.Conv2D(64, (3, 3), activation='relu'),
  layers.MaxPooling2D(2, 2),
  layers.Conv2D(64, (3, 3), activation='relu'),
  layers.MaxPooling2D(2, 2),
  layers.Flatten(),
  layers.Dense(1024, activation='relu'),
  layers.Dense(num_classes, activation='softmax')

])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_22 (Rescaling)    (None, 64, 64, 3)         0         
                                                                 
 conv2d_99 (Conv2D)          (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_99 (MaxPooli  (None, 31, 31, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_100 (Conv2D)         (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_100 (MaxPool  (None, 14, 14, 64)        0         
 ing2D)                                                          
                                                                 
 conv2d_101 (Conv2D)         (None, 12, 12, 64)      

In [ ]:
#determine num of images cropped to faces
image_count = 69540
skipped = []
incl_img = 0
# Load the DNN Face Detector model
from cv2 import dnn
prototxt = '/content/drive/MyDrive/ECE50024Challenge/deploy.prototxt.txt'
caffemodel='/content/drive/MyDrive/ECE50024Challenge/res10_300x300_ssd_iter_140000.caffemodel'
face_detector =  cv2.dnn.readNetFromCaffe( prototxt, caffemodel)

x_train = []
for k in range(image_count):
  if k < 10000:
    folder = '10000/'
  if k >= 10000 and k < 20000:
    folder = '20000/'
  if k >= 20000 and k < 30000:
    folder = '30000/'
  if k >= 30000 and k < 40000:
    folder = '40000/'
  if k >= 40000 and k < 50000:
    folder = '50000/'
  if k >= 50000 and k < 60000:
    folder = '60000/'
  if k >= 60000:
    folder = '69540/'
  images = folder+str(k)+'.jpg'
  img = cv2.resize(cv2.imread('/content/drive/MyDrive/ECE50024Challenge/train_faces/'+images), (64, 64))

  #only include the faces it's confident in
  # Get the height and width of the input image
  (h, w) = img.shape[:2]

  # Preprocess the image by resizing it and converting it to a blob
  blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

  # Feed the blob as input to the DNN Face Detector model
  face_detector.setInput(blob)
  detections = face_detector.forward()

  # Loop over the detections and draw a rectangle around each face
  confidence = detections[0, 0, 0, 2]

  # Filter out weak detections
  if confidence > 0.5:
    img = tf.convert_to_tensor(img, dtype=tf.float32)
    x_train.append(img)
    incl_img += 1
  else:
    skipped.append(k)
  print(k)

Streaming output truncated to the last 5000 lines.
64540
64541
64542
64543
64544
64545
64546
64547
64548
64549
64550
64551
64552
64553
64554
64555
64556
64557
64558
64559
64560
64561
64562
64563
64564
64565
64566
64567
64568
64569
64570
64571
64572
64573
64574
64575
64576
64577
64578
64579
64580
64581
64582
64583
64584
64585
64586
64587
64588
64589
64590
64591
64592
64593
64594
64595
64596
64597
64598
64599
64600
64601
64602
64603
64604
64605
64606
64607
64608
64609
64610
64611
64612
64613
64614
64615
64616
64617
64618
64619
64620
64621
64622
64623
64624
64625
64626
64627
64628
64629
64630
64631
64632
64633
64634
64635
64636
64637
64638
64639
64640
64641
64642
64643
64644
64645
64646
64647
64648
64649
64650
64651
64652
64653
64654
64655
64656
64657
64658
64659
64660
64661
64662
64663
64664
64665
64666
64667
64668
64669
64670
64671
64672
64673
64674
64675
64676
64677
64678
64679
64680
64681
64682
64683
64684
64685
64686
64687
64688
64689
64690
64691
64692
64693
64694
64695
64696
64697
6

In [ ]:
# get int values for the classes
with open('/content/drive/MyDrive/ECE50024Challenge/category.csv', mode ='r')as file:
  train_labels_num = csv.reader(file)
  k = 0
  c_num = np.zeros([num_classes,1])
  name_num = [] #Class name
  for lines in train_labels_num:
    if lines[0] != "":
        c_num[k] = lines[0]
        name_num.append(lines[1])
        k += 1

# Get labels for the data
k = 0
c = np.zeros([incl_img,1])
with open('/content/drive/MyDrive/ECE50024Challenge/train.csv', mode ='r')as file:
  train_labels = csv.reader(file)
  for lines in train_labels:
    if lines[0] != "":
      if int(lines[0]) not in skipped:
        #if lines[1] == images:
        index = name_num.index(lines[2])
        c[k] = c_num[index]
        k += 1
#assemble training sets
y_train = keras.utils.to_categorical(c, num_classes)

In [ ]:
print(np.shape(y_train))
print(np.shape(x_train))
print(incl_img) #check that proper number of confident images are included

(66232, 100)
(66232, 64, 64, 3)
66232


In [ ]:
x_train = tf.stack(x_train, axis=0)

In [ ]:
#run model
batch_size = 32
epochs = 6

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, validation_split=0.2)

Epoch 1/6
1656/1656 [==============================] - 13s 7ms/step - loss: 3.8905 - accuracy: 0.1348 - val_loss: 3.3192 - val_accuracy: 0.2483
Epoch 2/6
1656/1656 [==============================] - 11s 6ms/step - loss: 2.8838 - accuracy: 0.3568 - val_loss: 2.7592 - val_accuracy: 0.3915
Epoch 3/6
1656/1656 [==============================] - 11s 6ms/step - loss: 2.2985 - accuracy: 0.4878 - val_loss: 2.4879 - val_accuracy: 0.4731
Epoch 4/6
1656/1656 [==============================] - 11s 6ms/step - loss: 1.8738 - accuracy: 0.5780 - val_loss: 2.3995 - val_accuracy: 0.5133
Epoch 5/6
1656/1656 [==============================] - 11s 6ms/step - loss: 1.5476 - accuracy: 0.6471 - val_loss: 2.3488 - val_accuracy: 0.5391
Epoch 6/6
1656/1656 [==============================] - 11s 6ms/step - loss: 1.2594 - accuracy: 0.7034 - val_loss: 2.4948 - val_accuracy: 0.5385


In [ ]:
#Assemble test data

# Load the DNN Face Detector model
from cv2 import dnn
prototxt = '/content/drive/MyDrive/ECE50024Challenge/deploy.prototxt.txt'
caffemodel='/content/drive/MyDrive/ECE50024Challenge/res10_300x300_ssd_iter_140000.caffemodel'
face_detector =  cv2.dnn.readNetFromCaffe( prototxt, caffemodel)
#face_detector = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")

# get the path/directory
folder_dir = "/content/drive/MyDrive/ECE50024Challenge/test/"
for images in os.listdir(folder_dir):
    # check if the image ends with jpg
    if (images.endswith(".jpg")):
        #Make images readable
        img=Image.open('/content/drive/MyDrive/ECE50024Challenge/test/'+images)
        imgCheck = cv2.imread('/content/drive/MyDrive/ECE50024Challenge/test/'+images)
        if img.mode != 'RGB' or imgCheck is None:
            img = img.convert(mode="RGB", colors=24)
            img.save('/content/drive/MyDrive/ECE50024Challenge/test_readable/'+images)
            img = cv2.imread('/content/drive/MyDrive/ECE50024Challenge/test_readable/'+images)
        else:
            img = cv2.imread('/content/drive/MyDrive/ECE50024Challenge/test/'+images)

        # Get the height and width of the input image
        (h, w) = img.shape[:2]

        # Preprocess the image by resizing it and converting it to a blob
        blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))

        # Feed the blob as input to the DNN Face Detector model
        face_detector.setInput(blob)
        detections = face_detector.forward()

        # Loop over the detections and draw a rectangle around each face
        confidence = detections[0, 0, 0, 2]

        # Filter out weak detections
        if confidence > 0.25:
            # Get the bounding box for the face
            box = detections[0, 0, 0, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            if startX > np.shape(img)[1] or startX < 0:
                startX = 0
            if startY > np.shape(img)[0] or startY < 0:
                startY = 0
            if endX > np.shape(img)[1] or endX < 0:
                endX = np.shape(img)[1]
            if endY > np.shape(img)[0] or endY < 0:
                endY = np.shape(img)[0]

            faces = img[startY:endY, startX:endX]

            #write cropped faces to separate folder
            faces = cv2.resize(faces, (64, 64)) #resize the images to the same size
            path = '/content/drive/MyDrive/ECE50024Challenge/test_faces'
            cv2.imwrite(os.path.join(path , images), faces)
            cv2.waitKey(0)
        else:
            print(images)
            startX = 0
            startY = 0
            endX = np.shape(img)[1]
            endY = np.shape(img)[0]
            faces = img[startY:endY, startX:endX]

            #write cropped faces to separate folder
            faces = cv2.resize(faces, (64, 64)) #resize the images to the same size
            path = '/content/drive/MyDrive/ECE50024Challenge/test_faces'
            cv2.imwrite(os.path.join(path , images), faces)
            cv2.waitKey(0)

4237.jpg
1749.jpg
4631.jpg
1826.jpg
3604.jpg
545.jpg
343.jpg
4036.jpg


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


2523.jpg
2324.jpg
4757.jpg
47.jpg
3010.jpg
3819.jpg
1997.jpg
4385.jpg
4390.jpg
1362.jpg
609.jpg
1868.jpg
2780.jpg
286.jpg
133.jpg
858.jpg
1658.jpg
1499.jpg


KeyboardInterrupt: 

In [ ]:
folder_dir = "/content/drive/MyDrive/ECE50024Challenge/test_faces/"
for images in os.listdir(folder_dir):
    faces = cv2.imread('/content/drive/MyDrive/ECE50024Challenge/test_faces/'+images)
    faces = cv2.resize(faces, (64, 64)) #resize the images to the same size
    path = '/content/drive/MyDrive/ECE50024Challenge/test_faces'
    cv2.imwrite(os.path.join(path , images), faces)
    cv2.waitKey(0)

In [ ]:
#determine num of images cropped to faces
image_count = len(os.listdir('/content/drive/MyDrive/ECE50024Challenge/test_faces'))
print(image_count)

#assemble testing sets

y_test = keras.utils.to_categorical(c, num_classes)

x_test = []

for k in range(image_count):
  image = str(k)+'.jpg'
  img = cv2.imread('/content/drive/MyDrive/ECE50024Challenge/test_faces/'+image)
  x_test.append(img)

x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)

4977


In [ ]:
#Get predicted labels
labels = model.predict(x_test)

156/156 [==============================] - 1s 3ms/step


In [ ]:
print(labels[0])
print(image_count)
pred = []
for k in range(image_count):
    classGuess = np.where(labels[k] == max(labels[k]))
    nameGuess = name_num[int(classGuess[0])]
    row = [str(k), nameGuess]
    pred.append(row)
print(pred)
with open('/content/drive/MyDrive/ECE50024Challenge/submission4.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    field = ["Id", "Category"]
    writer.writerow(field)
    for k in range(image_count):
        writer.writerow(pred[k])

[5.56987107e-01 5.89340016e-05 6.55825343e-03 4.57884835e-06
 1.71578734e-03 4.10539098e-03 1.79289327e-05 6.17939467e-03
 2.07331008e-03 7.13660056e-03 7.55882593e-06 2.46202736e-03
 5.74835667e-06 4.22119301e-05 1.71817703e-06 2.13936684e-04
 2.44898984e-05 4.27338034e-02 3.22538824e-03 2.78290566e-02
 2.15377827e-06 1.26856327e-01 1.32415997e-04 8.49759963e-05
 2.28970801e-03 9.78572007e-06 1.76804955e-04 1.00009620e-05
 9.33497329e-04 5.04810851e-05 3.38064367e-03 2.27139099e-04
 2.58974754e-03 1.73806493e-05 5.07341447e-10 1.72184809e-06
 8.27333579e-09 9.07624963e-06 2.77323445e-04 2.51535210e-04
 1.00859620e-06 4.62926552e-03 2.23806655e-05 1.97383194e-08
 1.56555004e-07 4.57562862e-07 6.97439653e-04 1.72583823e-04
 3.40062165e-08 9.50230827e-08 3.35738249e-02 2.67221052e-02
 1.38027534e-10 2.01364746e-03 4.09771189e-07 7.82035962e-02
 7.34912931e-09 4.00361605e-04 1.35980781e-05 8.61100736e-04
 1.87615797e-06 8.61046065e-05 2.60241888e-03 2.11809620e-05
 9.48302059e-09 1.291326

<ipython-input-49-5b656d526696>:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  nameGuess = name_num[int(classGuess[0])]


[['0', 'Art Carney'], ['1', 'Amy Ryan'], ['2', 'Adrien Brody'], ['3', 'Adriana Barraza'], ['4', 'Albert Brooks'], ['5', 'Alec Baldwin'], ['6', 'Albert Brooks'], ['7', 'Adele'], ['8', 'Ava Gardner'], ['9', 'Alex Pettyfer'], ['10', 'Anna Faris'], ['11', 'Anthony Perkins'], ['12', 'Bill Hader'], ['13', 'Barabara Palvin'], ['14', 'Benjamin Bratt'], ['15', 'Berenice Bejo'], ['16', 'Alessandra Ambrosio'], ['17', 'Alan Arkin'], ['18', 'Akemi Darenogare'], ['19', 'Alan Arkin'], ['20', 'Ashley Graham'], ['21', 'Ashley Greene'], ['22', 'Benedict Cumberbatch'], ['23', 'Adele'], ['24', 'Andie MacDowell'], ['25', 'Alan Arkin'], ['26', 'Arnold Schwarzenegger'], ['27', 'Ben Affleck'], ['28', 'Andrew Lincoln'], ['29', 'Anna Friel'], ['30', 'Beyonce Knowles'], ['31', 'Ava Gardner'], ['32', 'Amy Schumer'], ['33', 'Al Pacino'], ['34', 'Alex Pettyfer'], ['35', 'Anjelica Huston'], ['36', 'Alex Pettyfer'], ['37', 'Ashley Greene'], ['38', 'Anna Paquin'], ['39', 'Alessandra Ambrosio'], ['40', 'Bill O Reilly']